In [ ]:
# 

# Install libraries

In [1]:
!pip install pyspark
!pip install matplotlib
!pip3 install pygad

# Import libraries

In [2]:
from pyspark.sql import SparkSession, functions as F, SQLContext
from pyspark.sql.functions import col
import random
from IPython.display import display, Image
import pygad
from pyspark.sql.functions import lower
from pyspark.sql.functions import when, lit, col
from pyspark.sql.functions import when

# Setup Spark

In [3]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

# Read data

In [4]:
df = spark.read.csv("../Data/pokemons.csv", header=True, inferSchema=True)

In [5]:
TypeData_df = spark.read.csv("../Data/TypeData.csv", header=True, inferSchema=True)

# Get 3 random pokemon

In [6]:
random.seed(42)

In [7]:
# Get IDs
pokemon_ids = df.select("id").distinct().rdd.map(lambda x: x[0]).collect()

# Randomly select 3 unique pokemon_ids
random_pokemon_ids = random.sample(pokemon_ids, 3)

# Filter the DataFrame based on the randomly selected pokemon_ids
random_pokemon = df.filter(col("id").isin(random_pokemon_ids))
random_pokemon.show()

+---+---------+---------+---------------+------------+-----+-------+---+---+---+-----+-----+-----+-----+------+------+--------------------+--------------------+
| id|     name|     rank|     generation|evolves_from|type1|  type2| hp|atk|def|spatk|spdef|speed|total|height|weight|           abilities|                desc|
+---+---------+---------+---------------+------------+-----+-------+---+---+---+-----+-----+-----+-----+------+------+--------------------+--------------------+
|314| illumise| ordinary| generation-iii|     nothing|  bug|   None| 65| 47| 75|   73|   85|   85|  430|     6|   177|oblivious tinted-...|With its sweet ar...|
|375|   metang| ordinary| generation-iii|      beldum|steel|psychic| 60| 75|100|   55|   80|   50|  420|    12|  2025|clear-body light-...|When two BELDUM f...|
|896|glastrier|legendary|generation-viii|     nothing|  ice|   None|100|145|130|   65|  110|   30|  580|    22|  8000|     chilling-neigh |Glastrier emits i...|
+---+---------+---------+---------

In [8]:
print("Opponents pokemons: ")
print()
for id in random_pokemon_ids:

    # Filter the random_pokemon DataFrame to get the details of the current pokemon_id
    pokemon_details = random_pokemon.filter(col("id") == id)\
        .select("id", "name", "rank", "generation", "evolves_from", "type1", "type2", "hp", "atk", "def", "spatk", "speed", "abilities", "desc")\
        .first()
    
    # Print the details in the specified format
    print("id:", pokemon_details["id"])
    print("name:", pokemon_details["name"])
    print("rank:", pokemon_details["rank"])
    print("generation:", pokemon_details["generation"])
    print("evolves_from:", pokemon_details["evolves_from"])
    print("type1:", pokemon_details["type1"])
    print("type2:", pokemon_details["type2"])
    print("hp:", pokemon_details["hp"])
    print("atk:", pokemon_details["atk"])
    print("def:", pokemon_details["def"])
    print("spatk:", pokemon_details["spatk"])
    print("speed:", pokemon_details["speed"])
    print("abilities:", pokemon_details["abilities"])
    # print("desc:", pokemon_details["desc"])
    desc = pokemon_details["desc"]
    print("desc:", end=" ")
    char_count = 0
    for word in desc.split():
        if char_count + len(word) > 64:
            print("\n\t", end="")
            char_count = 0
        print(word, end=" ")
        char_count += len(word) + 1  # Add 1 for the space between words
    print()

    
    # Convert the ID to a string with leading zeros if necessary
    pokemon_id_str = str(id).zfill(3)

    # Generate the URL for the Pokémon image
    url = f"https://assets.pokemon.com/assets/cms2/img/pokedex/full/{pokemon_id_str}.png"
    
    # Display the image
    display(Image(url=url))


Opponents pokemons: 

id: 896
name: glastrier
rank: legendary
generation: generation-viii
evolves_from: nothing
type1: ice
type2: None
hp: 100
atk: 145
def: 130
spatk: 65
speed: 30
abilities: chilling-neigh 
desc: Glastrier emits intense cold from its hooves. It’s also a 
	belligerent Pokémon—anything it wants, it takes by force. 


id: 375
name: metang
rank: ordinary
generation: generation-iii
evolves_from: beldum
type1: steel
type2: psychic
hp: 60
atk: 75
def: 100
spatk: 55
speed: 50
abilities: clear-body light-metal 
desc: When two BELDUM fuse together, METANG is formed. The brains of 
	the BELDUM are joined by a magnetic nervous system. By linking 
	its brains magnetically, this POKéMON generates strong 
	psychokinetic power. 


id: 314
name: illumise
rank: ordinary
generation: generation-iii
evolves_from: nothing
type1: bug
type2: None
hp: 65
atk: 47
def: 75
spatk: 73
speed: 85
abilities: oblivious tinted-lens prankster 
desc: With its sweet aroma, it guides Volbeat to draw signs with light 
	in the night sky. 


In [9]:

def get_n_row(df, number_in_df):
    num_rows = df.count()
    nth_row_df = df.limit(num_rows).collect()[num_rows - number_in_df]
    return spark.createDataFrame([nth_row_df], schema=df.schema)

def create(primary_type):
    # 1) Locate the type1 in TypeData based on primary_type
    TypeData_df_2 = TypeData_df.filter(TypeData_df.Type == primary_type.first()["type1"])
    # 2) Shorten df
    df_updated = df.select("id", "name", "type1", "hp", "atk").withColumn("bns", lit(None))
    # 3) Iterate over dataframe
    for column in TypeData_df_2.columns:
        df_updated = df_updated.withColumn("bns", when(col("type1") == column, TypeData_df_2.first()[column]).otherwise(df_updated["bns"]))
    # 4) Multiply value of column type with HP of orignal dataset
    return df_updated.withColumn("hp", col("hp") * col("bns")).drop("bns")


In [10]:
pokemon_1 = get_n_row(df=random_pokemon, number_in_df=1) # returns pyspark dataframe
pokemon_2 = get_n_row(df=random_pokemon, number_in_df=2) # returns pyspark dataframe
pokemon_3 = get_n_row(df=random_pokemon, number_in_df=3) # returns pyspark dataframe

all_enemies_of_pk1 = create(pokemon_1) # returns pyspark dataframe
#all_enemies_of_pk1.show()
all_enemies_of_pk2 = create(pokemon_2) # returns pyspark dataframe
#all_enemies_of_pk2.show()
all_enemies_of_pk3 = create(pokemon_3) # returns pyspark dataframe
#all_enemies_of_pk3.show()

# Hill Climbing

In [ ]:
attributes = ['hp', 'atk', 'def', 'spatk', 'spdef', 'speed']

# Calculate the total attribute score for each Pokémon in random_pokemon
random_pokemon_attributes = random_pokemon.selectExpr('id', 'hp + atk + def + spatk + spdef + speed AS total_score')

# Iterate through each attribute and compare with pokemon_df to find the top competitors
top_competitors = df
for attr in attributes:
    top_competitors = top_competitors.join(random_pokemon_attributes, random_pokemon_attributes.id == top_competitors.id, 'inner') \
                                     .withColumn('max_' + attr, col('df.' + attr) > col('random_pokemon_attributes.' + attr)) \
                                     .filter(col('max_' + attr) == True) \
                                     .drop('max_' + attr, 'random_pokemon_attributes.id')

# Show the top competitors
top_competitors.show()

In [ ]:
# Import necessary libraries
from pyspark import SparkContext
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def hill_climbing(f, x0, iter=1000):
    x = x0  # initial solution
    cur_iter = 0
    while iter >= cur_iter:
        # print(f"cur_iter: {cur_iter}, best: {x}")
        neighbors = generate_neighbors(x)  # generate neighbors of x
        # find the neighbor with the highest function value
        best_neighbor = max(neighbors, key=f)
        if f(best_neighbor) <= f(x):  # if the best neighbor is not better than x, stop
            return x, cur_iter
        x = best_neighbor  # otherwise, continue with the best neighbor
        cur_iter+=1

In [ ]:
# Define the gradient descent function
def gradient_descent(x, learning_rate=0.1, max_iter=1000, tolerance=10E-13):
    iteration = 0
    while iteration < max_iter:
        gradient = 2*x  # Gradient of the quadratic function -2*x
        step = learning_rate * gradient
        x -= step
        if abs(step) < tolerance:
            break
        iteration += 1
    return x, iteration

In [ ]:
# Define the function to optimize
def quadratic_function(x):
    return -x**2  # example function, we're maximizing, hence the negative sign

# Define a function to generate neighbors
def generate_neighbors(x):
    return [x - 0.1, x, x + 0.1]  # Simple neighbor generation for demonstration

In [ ]:

# Run hill climbing
initial_solution = 90
best_solution, num_of_iter = hill_climbing(quadratic_function, initial_solution)
print(f"Best solution found: {best_solution}, number of iterations: {num_of_iter}")

# Visualize the optimization process
x_values = np.linspace(-10, 10, 100)
y_values = [quadratic_function(x) for x in x_values]

plt.plot(x_values, y_values, label='Function')
plt.scatter(best_solution, quadratic_function(best_solution), color='red', label='Optimal Solution')
plt.title('Hill Climbing Optimization')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Run gradient descent
initial_solution = 2  # Initial value for gradient descent
best_solution, num_of_iter = gradient_descent(initial_solution)
print(f"Best solution found: {best_solution}, number of iterations: {num_of_iter}")

# Visualize the optimization process
x_values = np.linspace(-10, 10, 100)
y_values = [quadratic_function(x) for x in x_values]

plt.plot(x_values, y_values, label='Function')
plt.scatter(best_solution_gradient, quadratic_function(best_solution_gradient), color='green', label='Optimal Solution (Gradient Descent)')
plt.title('Gradient Descent Optimization')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.grid(True)
plt.show()
